* Part a,b


In [ ]:
# Import required libraries
from gurobipy import Model, GRB, quicksum
import pandas as pd

# Load datasets
farms = pd.read_csv('/Users/Sam/Downloads/farms.csv')
processing = pd.read_csv('/Users/Sam/Downloads/processing.csv')
centers = pd.read_csv('/Users/Sam/Downloads/centers.csv')

# Extract relevant columns dynamically
transport_cost_farm_to_plant = [col for col in farms.columns if 'Transport_Cost_To_Plant' in col]
transport_cost_plant_to_center = [col for col in processing.columns if 'Transport_Cost_To_Center' in col]


In [29]:
# Initialize the Gurobi optimization model
model = Model('BioAgri_Optimization')

# Decision variables for transportation
x = {}  # Amount transported from farms to processing facilities
y = {}  # Amount transported from processing facilities to centers

# Define decision variables
for i in farms.index:
    for j in range(len(transport_cost_farm_to_plant)):
        x[i, j] = model.addVar(vtype=GRB.CONTINUOUS, name=f"x_{i}_{j}")

for j in range(len(transport_cost_plant_to_center)):
    for k in centers.index:
        y[j, k] = model.addVar(vtype=GRB.CONTINUOUS, name=f"y_{j}_{k}")



In [30]:
# Objective function to minimize total costs
transport_cost = quicksum(
    farms.loc[i, transport_cost_farm_to_plant[j]] * x[i, j]
    for i in farms.index for j in range(len(transport_cost_farm_to_plant))
)

process_cost = quicksum(
    processing.loc[j, 'Processing_Cost_Per_Ton'] * quicksum(y[j, k] for k in centers.index)
    for j in processing.index
)

distribution_cost = quicksum(
    processing.loc[j, transport_cost_plant_to_center[k]] * y[j, k]
    for j in processing.index for k in centers.index
)

model.setObjective(transport_cost + process_cost + distribution_cost, GRB.MINIMIZE)



In [31]:
farms.head()

,Farm_ID,Bio_Material_Capacity_Tons,Quality,Cost_Per_Ton,Transport_Cost_To_Plant_1,Transport_Cost_To_Plant_2,Transport_Cost_To_Plant_3,Transport_Cost_To_Plant_4,Transport_Cost_To_Plant_5,Transport_Cost_To_Plant_6,...,Transport_Cost_To_Plant_9,Transport_Cost_To_Plant_10,Transport_Cost_To_Plant_11,Transport_Cost_To_Plant_12,Transport_Cost_To_Plant_13,Transport_Cost_To_Plant_14,Transport_Cost_To_Plant_15,Transport_Cost_To_Plant_16,Transport_Cost_To_Plant_17,Transport_Cost_To_Plant_18
0,Farm_1,478,2,127.46,2.055668,1.803083,1.069480,2.714386,2.612023,2.855007,...,1.139784,2.878670,1.471288,2.533115,1.569742,2.011931,2.966161,2.995096,2.430443,1.141342
1,Farm_2,308,2,137.42,2.575862,1.600978,1.639759,1.096792,1.506415,1.916566,...,2.280061,1.737035,2.789046,2.438423,2.504946,1.444706,2.159316,1.647811,1.073368,2.081945
2,Farm_3,516,3,189.20,2.680801,2.606527,2.193316,1.297035,1.497858,2.318891,...,2.878520,1.107940,1.368530,2.763425,1.061211,2.089573,1.002174,2.028174,1.858640,2.784677
3,Farm_4,367,1,66.23,2.296063,1.544005,2.209604,1.827935,2.180762,2.828915,...,1.003864,2.525939,1.426121,2.279124,1.566623,1.281170,2.801313,2.759361,2.508943,2.359971
4,Farm_5,499,1,86.06,1.677500,2.445887,1.960252,1.966784,1.140236,1.180227,...,2.451542,2.407016,1.542126,1.134395,1.997267,2.739380,2.140476,1.710534,2.166201,2.439978


In [32]:
processing.head()

,Processing_Plant_ID,Region,Capacity_Tons,Processing_Cost_Per_Ton,Transport_Cost_To_Center_1,Transport_Cost_To_Center_2,Transport_Cost_To_Center_3,Transport_Cost_To_Center_4,Transport_Cost_To_Center_5,Transport_Cost_To_Center_6,...,Transport_Cost_To_Center_93,Transport_Cost_To_Center_94,Transport_Cost_To_Center_95,Transport_Cost_To_Center_96,Transport_Cost_To_Center_97,Transport_Cost_To_Center_98,Transport_Cost_To_Center_99,Transport_Cost_To_Center_100,Transport_Cost_To_Center_101,Transport_Cost_To_Center_102
0,Plant_1,South,23717,4.274030,3.971489,2.615586,3.787975,3.385563,2.787156,2.330649,...,3.561992,2.670984,2.789733,2.145113,3.473826,2.978898,3.269141,2.599826,3.459213,3.653848
1,Plant_2,North,24413,4.674415,2.427791,2.578013,3.932194,2.400033,2.966497,3.870792,...,2.486606,2.725878,3.216768,2.709440,3.387974,2.977372,3.344813,2.363654,2.500414,2.312634
2,Plant_3,South,24563,7.165191,3.195770,2.937076,3.770210,3.946560,3.580432,3.737009,...,3.897593,3.012992,2.080397,2.054405,2.673700,2.155035,3.359755,2.891503,3.448340,2.687626
3,Plant_4,West,23238,6.593492,3.560335,3.978358,2.016401,2.699995,2.954555,3.728826,...,2.714315,2.721939,3.523792,3.106367,2.208035,2.381548,2.530702,2.488734,3.150051,3.449942
4,Plant_5,South,24486,6.279302,2.677966,3.777574,3.282922,3.691734,3.566555,3.386549,...,3.303730,3.566937,2.199177,2.423002,3.147445,2.641945,2.660690,3.055688,2.218378,2.031390


In [33]:
centers.head()

,Center_ID,Requested_Demand_Tons,Region
0,Center_1,82,West
1,Center_2,348,South
2,Center_3,464,North
3,Center_4,161,South
4,Center_5,340,West


In [34]:
# Ensure total supply from each farm does not exceed its capacity
for i in farms.index:
    model.addConstr(
        quicksum(x[i, j] for j in range(len(transport_cost_farm_to_plant))) <= farms.loc[i, 'Bio_Material_Capacity_Tons'],
        name=f"supply_constraint_{i}"
    )


In [35]:
# Ensure total processing at each facility does not exceed its capacity
for j in processing.index:
    # Balance input and output at processing facilities
    model.addConstr(
        quicksum(x[i, j] for i in farms.index) == quicksum(y[j, k] for k in centers.index),
        name=f"processing_balance_{j}"
    )
    # Limit processing capacity
    model.addConstr(
        quicksum(y[j, k] for k in centers.index) <= processing.loc[j, 'Capacity_Tons'],
        name=f"processing_capacity_{j}"
    )


In [36]:
# Ensure total fertilizer delivered to each center meets its demand
for k in centers.index:
    model.addConstr(
        quicksum(y[j, k] for j in processing.index) == centers.loc[k, 'Requested_Demand_Tons'],
        name=f"demand_constraint_{k}"
    )


In [37]:
# Optimize the model
model.optimize()

# Check if the solution is optimal
if model.status == GRB.OPTIMAL:
    print(f"Optimal Cost: {model.objVal}")
    for v in model.getVars():
        print(f"{v.varName}: {v.x}")
else:
    print("No optimal solution found.")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 387 rows, 14886 columns and 14472 nonzeros
Model fingerprint: 0x9aa9ecbc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 3e+04]
Presolve removed 0 rows and 8568 columns
Presolve time: 0.00s
Presolved: 387 rows, 6318 columns, 14472 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9457439e+05   3.627500e+03   0.000000e+00      0s
     129    2.2543337e+05   0.000000e+00   0.000000e+00      0s

Solved in 129 iterations and 0.01 seconds (0.01 work units)
Optimal objective  2.254333713e+05
Optimal Cost: 225433.3713483548
x_0_0: 0.0
x_0_1: 0.0
x_0_2: 0.0
x_0_3: 0.0
x_0_4: 0.0
x_0_5: 0.0
x_0_6: 0.0
x_0_7: 0.0
x_0_8: 478.0
x_0_9: 0.0
x_0_10: 0.0
x_0_1

In [38]:
# Extract results and save them
results = []
if model.status == GRB.OPTIMAL:
    for v in model.getVars():
        results.append({'Variable': v.varName, 'Value': v.x})
    # Convert to DataFrame for better readability
    results_df = pd.DataFrame(results)
    results_df.to_csv('/Users/Sam/Downloads/optimization_results.csv', index=False)
    print("Results saved to /Users/Sam/Downloads/optimization_results.csv")


Results saved to /Users/Sam/Downloads/optimization_results.csv


*
(a) How many sources of costs must be considered? How many decision variables are there?
Sources of Costs:

从农场到加工设施的运输成本。
加工设施的加工成本。
从加工设施到零售中心的运输成本。
共 3 个成本来源。
Decision Variables:

𝑥
[
𝑖
,
𝑗
]
x[i,j]：从农场 
𝑖
i 到加工设施 
𝑗
j 的运输量。
𝑦
[
𝑗
,
𝑘
]
y[j,k]：从加工设施 
𝑗
j 到零售中心 
𝑘
k 的运输量。
根据结果：模型有 14,886 个决策变量，即模型中的列数。

*
(b) Using Gurobi, what is the minimum cost of the transportation and procurement plan?
Optimal Cost:
模型计算的最优成本为 225,433.37。